In [1]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from keras.layers import (LSTM, Activation, Dense, Dropout,
                          Embedding, GlobalMaxPool1D, Input)
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from run_binary_classifier import _load_comments, run

/home/zvonimir/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def create_model(max_features):
    inp = Input(shape=(200, ))
    embed_size = 128
    x = Embedding(max_features, embed_size)(inp)
    x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
    return model

In [3]:
comments_X, comments_y = _load_comments('../../../data/train_binary_labels.csv')

In [4]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(comments_X))
comments_X = tokenizer.texts_to_sequences(comments_X)
comments_X = pad_sequences(comments_X, maxlen=200)

In [5]:
comments_X_train, comments_X_test, comments_y_train, comments_y_test = train_test_split(comments_X, comments_y, train_size=0.7, random_state=1)

/home/zvonimir/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
model = create_model(comments_X_train.shape[0])

In [7]:
hist = model.fit(comments_X_train, comments_y_train, batch_size=64, epochs=1, validation_split=0.2)

Train on 18171 samples, validate on 4543 samples
Epoch 1/1
18171/18171 [==============================] - 130s 7ms/step - loss: 0.3685 - acc: 0.8269 - val_loss: 0.2220 - val_acc: 0.9128


In [8]:
print('\n=================  Classification report  =================')

predictions = model.predict(comments_X_test, batch_size=64)
predictions[predictions > 0.5] = 1
predictions[predictions <= 0.5] = 0

print(classification_report(comments_y_test, predictions))


=================  Classification report  =================
             precision    recall  f1-score   support

          0       0.88      0.94      0.91      4823
          1       0.94      0.88      0.91      4912

avg / total       0.91      0.91      0.91      9735

